[Reference](https://medium.com/codex/visualising-satellite-data-with-pydeck-and-streamlit-to-discover-possible-forest-fires-worldwide-7cb4f25351ba)

# Retrieving the dataset


In [ ]:
import pandas as pd
import urllib.request
import os

DATA_URL = "https://queimadas.dgi.inpe.br/home/downloadfile?path=%2Fapp%2Fapi%2Fdata%2Fdados_abertos%2Ffocos%2FMensal%2Ffocos_abertos_mensal_202208.csv"
DATA_FILE = 'data\ForestFires202208.csv'
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

df = pd.read_csv('data\ForestFires202208.csv', encoding='utf-8', low_memory=False, date_parser=pd.to_datetime)
df = df[['lat', 'lon', 'data_hora_gmt', 'satelite', 'pais']]

df.to_csv('data\ForestFires.csv', index=False)
os.remove('data\ForestFires202208.csv')

# Creating our Streamlit application


In [ ]:
import streamlit as st
import pandas as pd
import datetime
import pydeck as pdk

st.set_page_config(layout="wide")

st.title("Forest Fires worldwide 🌳🔥")

# Importing and formatting the dataset


In [ ]:
@st.cache(persist = True)
def importing_dataset ():
    # Passing the file to df_heatSpots dataframe and formating the values types
    df_heatSpots = pd.read_csv('data/ForestFires.csv', encoding='utf-8')

    df_heatSpots[['satelite']] = df_heatSpots[['satelite']].astype('category')
    df_heatSpots[['pais']] = df_heatSpots[['pais']].astype('category')
    df_heatSpots[['data_hora_gmt']] = df_heatSpots[['data_hora_gmt']].astype('category')

    floats = df_heatSpots.select_dtypes(include=['float64']).columns.tolist()
    df_heatSpots[floats] = df_heatSpots[floats].astype('float32')

    df_heatSpots['data_hora_gmt'] = pd.to_datetime(df_heatSpots['data_hora_gmt'])

    return df_heatSpots

In [ ]:
def choosing_variables():
    # Date selection
    with st.sidebar:
        st.sidebar.markdown("**First select the data range you want to analyze:** 👇")
        date = st.date_input("Choose a August 2022 date",datetime.date(2022, 8, 6))
    
        # Country selection
        all_options_country = df_heatSpots['pais'].unique()
        select_country = st.multiselect("Country options (Leave blank to allow all countries)", all_options_country, ['Brasil'])

        if len(select_country) > 0:
            temp_select_country = select_country
        else:
            temp_select_country = all_options_country

        # Satellite selection
        all_options_satellite = df_heatSpots['satelite'].unique()
        select_satellite = st.multiselect("satellite options (Leave blank to allow all satellites)", all_options_satellite, ['NOAA-20'])

        if len(select_satellite) > 0:
            temp_select_satellite = select_satellite
        else:
            temp_select_satellite = all_options_satellite

    return date, temp_select_country, temp_select_satellite

# Querying the heat spots dataframe


In [ ]:
df_heatSpots = importing_dataset()

# Defining initial date country and satellite variables
date = datetime.datetime.strptime('19082022', "%d%m%Y").date()
temp_select_country = ['Brasil']
temp_select_satellite = ['NOAA-20']

# Hidding the possible options
date, temp_select_country, temp_select_satellite = choosing_variables()

# Querying the df_heatSpots dataframe
df_heatSpots = df_heatSpots[(df_heatSpots['data_hora_gmt'].dt.date == date) & 
                    (df_heatSpots['pais'].isin(temp_select_country)) &
                    (df_heatSpots['satelite'].isin(temp_select_satellite))]

# Setting the Pydeck map


In [ ]:
# Defining the Latitude and Longite as 0 to centre the map
lat0=0
lon0=0

# Set the viewport location
view_state = pdk.ViewState(latitude=lat0, longitude=lon0, zoom=1.9, bearing=0, pitch=45)

In [ ]:
# Define a layer to display on a map
layer = pdk.Layer(
    "ColumnLayer",
    df_heatSpots,
    get_position='[lon, lat]',
    elevation_scale=50,
    pickable=True,
    elevation_range=[50, 500],
    get_fill_color=[180, 0, 200, 140],
    extruded=True,
    radius=25,
    coverage=50,
    auto_highlight=True,
)

In [ ]:
# Generating tooltip for each generated point in the map with the related satellite
tooltip = {
    "html": "satellite: <b>{satellite}</b>",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

# Rendering the map 
st.pydeck_chart(pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style='mapbox://styles/mapbox/dark-v10',
    tooltip=tooltip
))